In [2]:
import cv2

In [3]:
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [5]:
model = cv2.dnn.readNet("C:\\Users\\sudat\\darknet\\data\\yolov4.weights", "C:\\Users\\sudat\\darknet\\cfg\\yolov4.cfg")

In [6]:
with open("C:\\Users\\sudat\\darknet\\data\\coco.names", "r") as f:
    classes = f.read().strip().split("\n")

In [7]:
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [8]:
path = "D:\\Work\\My work\\IT sem 1\\sem 3\\Object detection\\Road traffic video for object recognition.mp4"

In [9]:
cap = cv2.VideoCapture(path)

cap.set(3, 640)
cap.set(4, 480)

False

In [10]:
def detect_object(image):
    height, width = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    model.setInput(blob)
    
    layer_names = model.getUnconnectedOutLayersNames()
    detections = model.forward(layer_names)
    
    conf_threshold = 0.90
    classIds = []
    confidences = []
    boxes = []
    objects_detected = []
    
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > conf_threshold:
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                w = int(obj[2] * width)
                h = int(obj[3] * height)
                
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                classIds.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                objects_detected.append((x, y, x + w, y + h))
                
    nmsThreshold = 0.3
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nmsThreshold)
    
    for i in indices:
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        
        class_id = classIds[i]
        confidence = confidences[i]
        
        color = (0, 255, 0)  # Green color
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        label = f"{classes[class_id]}: {confidence:.2f}"
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        
        max_display_width = 1000 
        if width > max_display_width:
            scaling_factor = max_display_width / width
        else:
            scaling_factor = 1.0
        
        display_image = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor)
        cv2.imshow("Object Detection", display_image) 

In [11]:
while True:
    ret, frame = cap.read()
    detect_object(frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



In [12]:
cap.release()
cv2.destroyAllWindows()